![banner](../logo/banner.png)

## 3.7 Salidas analogicas (PWM)

Los pines PWM son utilizados como un reemplazo a un DAC, es decir, se toma como una salida analogica. Los pines que se pueden utilizar como salida analogica (PWM) son las que tienen el simbolo `~`, en la placa.


![board_pwm](imgs/board_pwm.png)

Los cuales son los pines:

Pines|
:-:|
3|
5|
6|
9|
10|
11|

Esto significa que podemos tener 6 salidas analogicas para poder controlar cualquier elemento que soporte este control.

## 3.7.1 Configuracion de salidas PWM

Vamos a realizar la configuracion basica del PWM

```python
from pyfirmata import Arduino, util
from time import sleep
 
PORT = 'COM#'
board = Arduino(PORT)

# Configuramos el pin que sera utilizado como PWM
pin_analog = board.get_pin('d:6:p')

# Mando el valor que necesito al pin 6 con un ciclo de trabajo del 60%
pin_analog.write(0.5)

```

**Ejemplo 1: Vamos a controlar la intensidad de un LED con una salida PWM**

![pwm basico](imgs/pwm_1.png)

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

# Configuramos el pin que sera utilizado como PWM
pin_analog = board.get_pin('d:3:p')

while True:

    for i in range(0, 101,3):
        pin_analog.write(i/100)
        sleep(0.2)
```

---
**Ejemplo 2: Vamos a controlar el intensidad de un LED con un 2 botones, uno que incremente y con otro que decremente la intesidad**

![pwm btn](imgs/pwm_btn.png)

---
**Ejemplo 3: Vamos a controlar el intensidad de un LED con potenciometro la variar su intensidad.**

![](imgs/pwm_adc_1.png)

## 3.7.2 Servomotor

El servomotor se mueve con base a una señal PWM, pero la libreria nos ahorra el trabajo para generar el desplazamiento.

El servomotor que utilizaremos el SG-90, es un servomotor basico.

![servo sg90](imgs/sg90.jpg)

Este motor necesita la siguiente señal para poder generar su desplazamiento:

![signal servo](imgs/pwm_servo_data.png)


### 3.7.2.1 Configurando Servo

Los pines que se pueden usar para el control de servo son los que le pertenecen al PWM.

Tenemos dos formas de controlar el Servo, una es haciendolo de manera manual y la otra usando la libreria directamente.

**Forma 1: manual**

```python
from pyfirmata import Arduino, util
from time import sleep

PORT = 'COM#'
board = Arduino(PORT)

# Configuramos el pin que sera utilizado como PWM
servo = board.get_pin('d:3:p')

while True:

    for i in range(0.002, 0.210,0.002):
        servo.write(i)
        
```
**Forma 2. Con la libreria**

```python
```

Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)

[Mecatrónica 85](https://mecatronica85.com/)

[fb/mecatronica85](https://www.facebook.com/mecatronica85)